In [1]:
import itertools
import string
import random
import json
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [2]:
def generate_row_letters(num_strings: int, max_length: int = 2):
    counter = 0
    for length in range(1, max_length + 1):
        if counter >= num_strings:
            break
        for item in itertools.product(string.ascii_uppercase, repeat=length):
            if counter >= num_strings:
                return
            yield "".join(item)
            counter += 1

In [9]:
def generate_plate(rows: int, cols: int):
    plate = np.random.randint(1, 100, size=(rows, cols))
    return plate

In [3]:
def _gen_filler(n_cols: int, has_empty: bool = True) -> list[float]:
    fill = []
    for _ in range(n_cols):
        if has_empty:
            fill.append(None)
        # bias towards blanks since most plates are surrounded by blanks
        r = random.randint(1, 6 if has_empty else 3)
        if r == 1:
            fill.append(random.random())
        elif r == 2:
            fill.append(random.randint(1, 1000))
        elif r == 3:
            res = ''.join(random.choices(string.ascii_lowercase + string.ascii_uppercase +
                             string.digits, k=random.randint(5, 50)))
            fill.append(res)
        else:
            fill.append(None)
    return fill
    

In [4]:
def create_nontabular_plate(
    n_rows: int,
    n_cols: int,
    offset: int,
    contents: str = 'Plate',
    n_filler: int = 10
) -> pd.DataFrame:
    cols_before = random.randint(1, n_filler)
    cols_after = random.randint(1, n_filler)
    tot_cols = n_cols + cols_before + cols_after
    rows_before = random.randint(1, n_filler)
    rows_after = random.randint(1, n_filler)
    # Add one for the header
    row_start = rows_before + offset + 1
    results = {'row_start': row_start, 
               'row_end': row_start + (n_rows - 1), # it's inclusive
               'col_start': cols_before, 
               'col_end': cols_before + (n_cols - 1), # it's inclusive
               'contents': contents}
    plate_rows = []
    letters = list(generate_row_letters(n_rows))
    # Generate junk rows before the header
    for row in range(rows_before):
        plate_rows.append(_gen_filler(tot_cols))
    # Generate the header which is a monotonically increasing sequence
    plate_header = (
        _gen_filler(cols_before) + list(range(1, n_cols + 1)) + _gen_filler(cols_after)
    )
    plate_rows.append(plate_header)
    # Fill out the rest of the plate
    for row in range(n_rows):
        data_row = (
            _gen_filler(cols_before - 1)
            + [letters[row]]
            # This is the plate contents
            + _gen_filler(n_cols, has_empty=False)
            + _gen_filler(cols_after)
        )
        plate_rows.append(data_row)
    for row in range(rows_after):
        plate_rows.append(_gen_filler(tot_cols))
    return pd.DataFrame(plate_rows), results

In [5]:
def create_plates(
    n_plates: int,
    n_rows: int,
    n_cols: int,
    return_dict: bool = False,
) -> pd.DataFrame | dict[str, pd.DataFrame]:
    plates: list[pd.DataFrame] = []
    results: list[dict] = []
    offset = 0
    for i in range(n_plates):
        plate_df, result = create_nontabular_plate(n_rows, n_cols, offset)
        offset += plate_df.shape[0]
        plates.append(plate_df)
        results.append(result)
    return {f"plate_{i}": plate_data for i, plate_data in enumerate(plates)} if return_dict else pd.concat(plates, ignore_index=True), results

In [6]:
def _get_plate_from_result_dict(df: pd.DataFrame, result_dict: dict) -> pd.DataFrame:
    row_start, row_end = result_dict['row_start'], result_dict['row_end'] + 1
    col_start, col_end = result_dict['col_start'], result_dict['col_end'] + 1
    proposed_plate = df.iloc[
        row_start:row_end,
        col_start:col_end,
    ]
    return proposed_plate

In [10]:
n_plates = 2
plates_df, results = create_plates(n_plates, 4, 6)
print(results)
_get_plate_from_result_dict(plates_df, results[0])

[{'row_start': 8, 'row_end': 11, 'col_start': 4, 'col_end': 9, 'contents': 'Plate'}, {'row_start': 19, 'row_end': 22, 'col_start': 7, 'col_end': 12, 'contents': 'Plate'}]


/var/folders/vy/cb58vztd6ldb8lmvkrzxxfy80000gp/T/ipykernel_5763/3468230474.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return {f"plate_{i}": plate_data for i, plate_data in enumerate(plates)} if return_dict else pd.concat(plates, ignore_index=True), results


,4,5,6,7,8,9
8,None,None,A,1,956,F1ajilfsxvnnuhyaDTRaqcHtEy4So1KFKtJe3k5KL53IF6pJ
9,None,None,B,574,RQGhiToKkXE5NaNOtwv7qKEb38iaOX,584
10,None,872,C,402,0.701383,250
11,None,None,D,348,0.023827,m7ruCPPjbSjVL3L


In [11]:
plates_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,None,XgW1yG2JziKl,None,0.831536,None,None,None,950,None,None,None,0.080083,None,0.972764,NaN,None,NaN,None,None,0.438795,None,UVIidz9GEzObkBTZ3Q5VwtiD,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
1,None,None,None,IRIBOFhzALkofBWwqpJ2vYlBITfPGg2wmQUbisMiKvo4Z,None,881,None,None,None,None,None,None,None,NaN,NaN,None,NaN,None,None,None,None,Kz7y1Bo1kSxXpYs6OC8tE55AcAJFAx1xWlCEa6oAYSO7G9,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,None,0.139817,None,578,None,None,None,5qsRyFt2ICMTLUxk,None,0.440834,None,233.0,NaN,None,NaN,0.665498,None,None,None,None,None,None,NaN,0.236294,NaN,NaN,NaN,NaN,NaN,NaN
3,None,955,None,None,None,None,None,vvo6c8d3uRKjMqfVVGoP7Re0m6Pby1,None,None,None,820,None,NaN,NaN,None,NaN,628,None,72,None,0.737257,None,0.509196,NaN,BGmdzi4MBDB9BYyYCalKAwwPzUJkwB5iMZt3u3nJM0r9wkQK,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,None,None,None,106,None,0.266888,None,None,None,478,None,NaN,NaN,904,NaN,79,None,None,None,0.611219,None,278,NaN,VGnvKB2ObtB6ewjY0mMVbkjWhyVNxO,NaN,NaN,NaN,NaN,NaN,NaN
5,None,None,None,None,None,None,None,None,None,2U41GGmCM,None,8mBBWFVIFrsB9S179QlBLFVVwpgR3CgSjnc,None,0.613069,NaN,0.209509,NaN,NXZqzJyEUe,None,None,None,615,None,VXearSrtExKHMjTTxhqvTQaGil2AGVzPwufIsM3DgRsxX,NaN,0.720037,NaN,NaN,NaN,NaN,NaN,NaN
6,None,None,None,664,None,None,None,None,None,0.67897,None,None,None,0.461165,NaN,None,NaN,None,None,mRuUQW0wzjuCV,None,528,None,wH7ssTVjc,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
7,None,None,None,0.841097,None,24,None,None,1,2,3,4,5,6.0,NaN,44,NaN,HQ1xIQxv1rZDjuXRZQZ,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
8,None,8ndV05sYjJdGxdqDP4zqiwj7ucMij,None,0.930407,None,None,A,1,956,F1ajilfsxvnnuhyaDTRaqcHtEy4So1KFKtJe3k5KL53IF6pJ,480,464,27,NaN,0.275999,None,958.0,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
9,None,None,None,None,None,None,B,574,RQGhiToKkXE5NaNOtwv7qKEb38iaOX,584,0.94756,315,0.261854,NaN,NaN,None,571.0,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
evals = []
for i in range(100):
    plates_df, results = create_plates(random.randint(1, 10), 4, 6)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})
for i in range(100):
    plates_df, results = create_plates(random.randint(1, 10), 8, 12)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})
for i in range(100):
    plates_df, results = create_plates(random.randint(1, 10), 16, 24)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})
for i in range(10):
    plates_df, results = create_plates(random.randint(1, 10), 32, 48)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})

In [108]:
with open('gen_evals.json', 'w') as f:
    json.dump(evals, f)

In [114]:
plates_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,None,0lGZsTtZqRq1YGyEHYGFhR0DW7bIfUNSxJlbjCPHQLfa11KwC,None,None,None,None,None,None,None,None,NaN,0.121591,None,966,None,NaN,None,112,None,None,None,0.983377,None,608.0,None,NaN,None,NaN,None,gZO1PtE0nKJ3lwzDGw,None,None,NaN,NaN
1,None,0.2955,None,l5sIozlzJfnOQ67xrmUzpiBVZoB7TD,None,0.269455,None,None,None,None,NaN,129,None,kVtQ5VBE49u,None,NaN,None,None,1,2,3,4,5,6.0,None,NaN,None,NaN,None,None,None,None,NaN,NaN
2,None,722,None,None,None,i6AstG22s8Cb88ktNduWfu2O4ZSMtT18ySfhFlK0d9t,None,None,None,1z3LMVH9UTx5OxbazLSJl8pgxjLLspp7zWZPrJwAf02Nnshd7,NaN,0.029341,None,None,None,NaN,A,0.020699,0.776257,oaqsnXhXqqFyvPybUFaapUftTdEroY303ahWMKxYBIZ,TryYzSOqwPejSMmr4DMJrEhoO55nbsTJ3SY,0.861957,lfBqcQsv2oMWoJCzLygxKIedwyQ7iXT1juIUDJThZdsB3r...,NaN,kmzohBqQcxtPHkJmhODI6Ov2joDNmBMY,NaN,None,NaN,None,None,None,None,NaN,NaN
3,None,0.702402,None,None,None,None,None,YTOxGPEGlSH7KlONI6uSzZyVj7PdJ3eVlf9fFALXmRgUdzcU,None,None,NaN,268,None,0.209087,None,0.4656,B,20us17NGMB5AHUn72EXfiEXupxTMXtyUL5xDUg3MKuYID,70,D28qisjvRiONwW9nKhGuQoHmgHxVppiLJhgg,0.683478,907,0.827364,NaN,None,NaN,None,NaN,None,None,None,None,NaN,NaN
4,None,None,None,None,None,None,None,None,None,None,NaN,None,None,478,None,NaN,C,41,xhHtGvH9bRmofTl6PT4uRMUdZpqCUrHl7nib72o,8,0.013126,0.337339,948,NaN,None,NaN,None,NaN,None,None,None,None,NaN,NaN
5,None,0.15484,None,None,None,None,None,None,None,945,NaN,538,None,KUk8wQH99CzjAR2t94GxwRMwLiXB7WRZhQDk29i7HBxm0i...,None,0.224276,D,340,66,Fv78o91,e32TAsxqeqtjGXrxl9cQc5,NE73bDert9O6LTTieu0pDwOJtYlPjJ78z1A,cHuOBrgAI7jgxAusDWMMUlOx5tRa9KRz4Vem,NaN,None,NaN,None,NaN,None,None,None,None,NaN,NaN
6,None,None,None,None,None,0.531886,None,None,None,0.342786,NaN,None,None,None,None,0.696387,None,DVg1fdLMN3vu,None,None,None,151,None,0.521799,None,308.000000,None,NaN,None,None,None,None,NaN,NaN
7,None,0.045334,None,None,None,None,None,None,None,None,NaN,bYGUODKj0l,None,None,None,546.0,None,None,None,IfBupSHirZ,None,None,None,0.12874,None,0.161239,None,0.758832,None,None,None,k98otDxWQYlNmX7Fr1uVsWi8y,NaN,NaN
8,None,0.439353,None,None,None,None,None,148,None,None,NaN,0.665736,None,0.716956,None,NaN,None,975,None,None,None,158,None,411.0,None,NaN,None,NaN,None,None,None,None,NaN,NaN
9,None,ooMs0vYObIIXBsYwkp,None,0.904097,None,eyODFr,None,None,None,NaN,NaN,NaN,None,None,None,None,None,337,NaN,252,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN,None,a6EurjfKbqRwNAsTHOw6LlU6yEoFL26LXPVofBm0
